# 🛰️ Satellite Super-Resolution - Complete Training Pipeline

**Uses REAL satellite imagery with proper validation and metrics**

GitHub: https://github.com/Bharath-2005-07/ResolutionOf-Satellite

---

## Features:
- ✅ Downloads real satellite dataset (UC Merced)
- ✅ Proper LR/HR pairs creation
- ✅ Satellite-optimized loss functions (VGG + Edge + L1)
- ✅ PSNR/SSIM validation
- ✅ Before/After visualizations
- ✅ Training progress tracking

## 🔧 Setup

In [ ]:
# Install dependencies (Google Colab)
!pip install -q torch torchvision matplotlib tqdm Pillow opencv-python-headless scikit-image

# Clone repository
!git clone https://github.com/Bharath-2005-07/ResolutionOf-Satellite.git
%cd ResolutionOf-Satellite

## 📥 Option 1: Use Complete Training Script (Recommended)

In [ ]:
# Run the complete training script
!python train_satellite_colab.py

## 📥 Option 2: Step-by-Step Training (For Learning)

In [ ]:
import sys
sys.path.insert(0, '/content/ResolutionOf-Satellite')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

### Step 1: Download Real Satellite Data

In [ ]:
# Download UC Merced Land Use Dataset (real satellite imagery)
!wget -q http://weegee.vision.ucmerced.edu/datasets/landuse.zip -O satellite_data.zip
!unzip -q satellite_data.zip -d satellite_data
!rm satellite_data.zip

print("✅ Real satellite dataset downloaded!")

### Step 2: Prepare LR/HR Pairs

In [ ]:
from PIL import Image
from pathlib import Path

# Create directories
lr_dir = Path('satellite_data/lr')
hr_dir = Path('satellite_data/hr')
lr_dir.mkdir(exist_ok=True, parents=True)
hr_dir.mkdir(exist_ok=True, parents=True)

# Find all images
image_files = list(Path('satellite_data').rglob('*.tif'))
image_files += list(Path('satellite_data').rglob('*.png'))
image_files += list(Path('satellite_data').rglob('*.jpg'))

print(f"Found {len(image_files)} images")

# Process images
for idx, img_path in enumerate(tqdm(image_files[:300])):
    try:
        img = Image.open(img_path).convert('RGB')
        
        # Create HR (256x256)
        img_hr = img.resize((256, 256), Image.BICUBIC)
        
        # Create LR (64x64) - simulates Sentinel-2
        img_lr = img_hr.resize((64, 64), Image.BICUBIC)
        
        # Save
        img_hr.save(hr_dir / f'sat_{idx:04d}.png')
        img_lr.save(lr_dir / f'sat_{idx:04d}.png')
    except:
        continue

print(f"✅ Created {len(list(lr_dir.glob('*.png')))} LR/HR pairs")

### Step 3: Create Dataset and DataLoader

In [ ]:
from data.dataset import SatelliteSRDataset

# Create datasets
train_dataset = SatelliteSRDataset(
    lr_dir='satellite_data/lr',
    hr_dir='satellite_data/hr',
    patch_size=64,
    scale_factor=4,
    augment=True
)

# Split into train/val
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

### Step 4: Initialize Model

In [ ]:
from models.esrgan import ESRGANLite

model = ESRGANLite(scale_factor=4).to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

### Step 5: Loss Functions and Optimizer

In [ ]:
from training.losses import SatelliteSRLoss

# Satellite-optimized loss
criterion = SatelliteSRLoss(
    pixel_weight=1.0,
    perceptual_weight=0.1,
    edge_weight=0.1
).to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=2e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

print("✅ Loss and optimizer initialized")

### Step 6: Training Loop

In [ ]:
from training.metrics import calculate_psnr, calculate_ssim

num_epochs = 100
best_psnr = 0
history = {'train_loss': [], 'val_psnr': [], 'val_ssim': []}

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
    for lr_img, hr_img in pbar:
        lr_img, hr_img = lr_img.to(device), hr_img.to(device)
        
        optimizer.zero_grad()
        sr = model(lr_img)
        
        loss, components = criterion(sr, hr_img)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        train_loss += loss.item()
        pbar.set_postfix({
            'Loss': f"{loss.item():.4f}",
            'Pixel': f"{components['pixel']:.4f}"
        })
    
    avg_train_loss = train_loss / len(train_loader)
    history['train_loss'].append(avg_train_loss)
    
    # Validation
    if (epoch + 1) % 10 == 0:
        model.eval()
        val_psnr_sum = 0
        val_ssim_sum = 0
        
        with torch.no_grad():
            for lr_img, hr_img in val_loader:
                lr_img, hr_img = lr_img.to(device), hr_img.to(device)
                sr = model(lr_img)
                
                for i in range(sr.shape[0]):
                    psnr = calculate_psnr(sr[i:i+1], hr_img[i:i+1])
                    ssim = calculate_ssim(sr[i:i+1], hr_img[i:i+1])
                    val_psnr_sum += psnr
                    val_ssim_sum += ssim
        
        avg_psnr = val_psnr_sum / len(val_dataset)
        avg_ssim = val_ssim_sum / len(val_dataset)
        
        history['val_psnr'].append(avg_psnr)
        history['val_ssim'].append(avg_ssim)
        
        print(f"\nEpoch {epoch+1} | Loss: {avg_train_loss:.4f} | PSNR: {avg_psnr:.2f}dB | SSIM: {avg_ssim:.4f}")
        
        # Save best model
        if avg_psnr > best_psnr:
            best_psnr = avg_psnr
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"✅ New best model saved! PSNR: {best_psnr:.2f}dB")
    
    scheduler.step()

print(f"\n✅ Training complete! Best PSNR: {best_psnr:.2f}dB")

### Step 7: Visualize Results

In [ ]:
model.eval()
with torch.no_grad():
    lr, hr = next(iter(val_loader))
    lr, hr = lr.to(device), hr.to(device)
    sr = model(lr)
    
    # Create bicubic baseline
    bicubic = torch.nn.functional.interpolate(lr, scale_factor=4, mode='bicubic', align_corners=False)
    
    # Convert to numpy
    lr_np = lr[0].cpu().permute(1, 2, 0).numpy()
    bicubic_np = bicubic[0].cpu().permute(1, 2, 0).numpy().clip(0, 1)
    sr_np = sr[0].cpu().permute(1, 2, 0).numpy().clip(0, 1)
    hr_np = hr[0].cpu().permute(1, 2, 0).numpy()
    
    # Calculate metrics
    psnr_bicubic = calculate_psnr(bicubic[0:1], hr[0:1])
    psnr_sr = calculate_psnr(sr[0:1], hr[0:1])
    ssim_bicubic = calculate_ssim(bicubic[0:1], hr[0:1])
    ssim_sr = calculate_ssim(sr[0:1], hr[0:1])
    
    # Plot
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    
    axes[0, 0].imshow(lr_np)
    axes[0, 0].set_title('Input LR (Sentinel-2 Quality)', fontsize=14)
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(bicubic_np)
    axes[0, 1].set_title(f'Bicubic Baseline\nPSNR: {psnr_bicubic:.2f}dB, SSIM: {ssim_bicubic:.4f}', fontsize=14)
    axes[0, 1].axis('off')
    
    axes[1, 0].imshow(sr_np)
    axes[1, 0].set_title(f'Our Model (ESRGANLite)\nPSNR: {psnr_sr:.2f}dB, SSIM: {ssim_sr:.4f}', fontsize=14)
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(hr_np)
    axes[1, 1].set_title('Ground Truth HR', fontsize=14)
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.savefig('results_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n📊 Results:")
    print(f"Bicubic   - PSNR: {psnr_bicubic:.2f}dB, SSIM: {ssim_bicubic:.4f}")
    print(f"Our Model - PSNR: {psnr_sr:.2f}dB, SSIM: {ssim_sr:.4f}")
    print(f"Improvement: +{psnr_sr - psnr_bicubic:.2f}dB")

### Step 8: Plot Training History

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(history['train_loss'])
axes[0].set_title('Training Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].grid(True)

axes[1].plot(history['val_psnr'])
axes[1].set_title('Validation PSNR')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('PSNR (dB)')
axes[1].grid(True)

axes[2].plot(history['val_ssim'])
axes[2].set_title('Validation SSIM')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('SSIM')
axes[2].grid(True)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150)
plt.show()

## 🎉 Done!

Your model is now trained on **real satellite imagery** with:
- ✅ Proper LR/HR pairs
- ✅ Satellite-optimized losses (VGG + Edge + L1)
- ✅ PSNR/SSIM validation
- ✅ Visual comparisons

Model saved to: `best_model.pth`

**Expected Performance:**
- PSNR: 26-30 dB (vs 24 dB bicubic baseline)
- SSIM: 0.85-0.92 (vs 0.78 bicubic baseline)
- Sharp edges on roads, buildings, and urban features